## This code is use to set and train the model an CHI@TC A100

In [ ]:
from chi import server, context, lease
import os

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@UC")

In [ ]:
l = lease.get_lease(f"llm_project45") # or llm_single_netID, or llm_multi_netID
l.show()

In [ ]:
username = 'project45'
s = server.Server(
    f"node-llm-{username}", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

In [ ]:
#associate_floating_ip()
s.associate_floating_ip()

In [ ]:
s.refresh()
s.check_connectivity()

In [ ]:
s.refresh()
s.show(type="widget")

In [ ]:
s.execute("git clone https://github.com/Phase1es2/ML-Sys-DevOps.git")

### set up docker with NVIDIA conatiner toolkit

In [ ]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")
s.execute("docker run hello-world")

In [ ]:
# get NVIDIA container toolkit 
s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
s.execute("sudo apt update")
s.execute("sudo apt-get install -y nvidia-container-toolkit")
s.execute("sudo nvidia-ctk runtime configure --runtime=docker")
s.execute("sudo systemctl restart docker")

In [ ]:
s.execute("docker run --rm --gpus all ubuntu nvidia-smi")

In [ ]:
s.execute("docker pull quay.io/jupyter/pytorch-notebook:cuda12-pytorch-2.5.1")

In [ ]:
s.execute("docker run -d -p 8888:8888 --gpus all --name torchnb quay.io/jupyter/pytorch-notebook:cuda12-pytorch-2.5.1")

In the local Terminal run:

``` ssh -L 8888:127.0.0.1:8888 -i ~/.ssh/id_rsa_chameleon cc@A.B.C.D```

In [ ]:
s.execute("docker logs torchnb")

looks for ```http://127.0.0.1:8888/lab?token=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
```

If need to monitor GPU usage:

In [ ]:
#run on node not in this jupyer
sudo apt update
sudo -y install nvtop